In [ ]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Priority Queues/Heaps


Today's agenda:

- Priority Queues/Heap

We saw two greedy algorithms that both construct the solution by evaluating a greedy choice. But how do we easily identify the choice? 

Often times, we can sort the solution choices by our greedy criterion. But this only works if we don't need to recompute the criterion each time we change the solution.

The *priority queue* is a tree-based data structure that mates well with greedy algorithms since it allows for efficient insertions, removals and updates of items. 

### Priority Queues

For simplicity we'll assume that we are always seeking the minimum-value element from the priority queue. The priority queue data structure needs to support some basic operations:

- *deleteMin*: Identify the element with minimum value and remove it. 

- *insert(x, s)*: insert a new element $x$ with initial value $s$.

Note that for the static case in which scores don't change, sorting took $O(n\log n)$ work and $O(\log^2 n)$ span.

This suggests that we should aim for $O(\log n)$ work per operation.



### The Heap Property

The *heap property* for a tree states that every node in the tree is smaller than either of its children. This means that the root of a tree with the heap property is always the minimum element. So for a binary tree:

<img src="heap_property.jpg" width="70%">

Notice that a binary heap is less restrictive than a binary search tree since the left and right subtrees can be swapped.

We've seen that binary trees that have all possible nodes have logarithmic depth. In lab you implement a *binary heap*, which has performance since it is an "almost-complete", and thus balanced, binary tree. 

Maintaining the heap property upon insertion or deletion requires time proportional to the depth of the tree because we can swap elements upward or downward, following the path from the modification either upward or downward.

### Meldable Heaps

We'll look at an alterntaive to maintaining the shape property. We will still use binary trees and maintain the heap property, but will not require them to be almost-complete. We will make use of the observation that really, heap operations just require the ability to combine or *meld* heaps efficiently:

- *deleteMin* needs to delete the root of the tree, and then somehow meld the left and right subtrees.
- *insert* is just the melding of the current tree and and a singleton tree.


Suppose we wish to meld two heaps $A$ and $B$, with $A$ smaller than $B$. To create a single tree $C$ from $A$ and $B$, we need to decide on the root. Suppose we let the root $r_A$ of the smaller tree $A$ be the new root. What do we do with the left and right subtrees $L_A$ and $L_B$ of $r_A$ and $B$? 

If we maintain the left subtree $L_A$ of $r_A$, we can meld $L_B$ and $B$ and make this the right subtree of $r_A$. 

<img src="meld_schematic.jpg" width="70%">

This defines a recursive procedure for melding two heaps:

<img src = "naive_meldable_heap_spec.png" width="40%">

<img src = "example_heap_meld.jpg" width="50%">

This is a well-defined procedure for melding two heaps, but as we can see in this example, we may actually obtain a very long right "spine" of the melded tree. Actually in the worst case we might take $\Theta(|A|+|B|)$ work! 

### Leftist Heaps

To address this imbalance in our approach, we can do some bookkeeping and use our flexibility in choosing how to orient subtrees left to right. Let $rank(x)$ be length of the right spine of starting at $x$. 

A **leftist heap** has the property that for any node $x$ in the heap, $rank(L(x)) \geq rank(R(x))$ where $L(x)$ is the left child of $x$ and $R(x)$ is the right child of $x$.

Of course, a leftist heap could be imbalanced on the left:

<img src="leftist_heap-unbalanced.jpg" width="20%">

Fortunately, keeping the $rank$ measure at every node will allow us to essentially balance the heap. The key idea is that since melding only recurses right, if we use meld to insert elements into the heap it will "balance out" the left bias of the leftist property.


<img src = "leftist_meldable_heap_spec.png" width="40%">

We maintain a rank at each leftist node, incrementing it and always guaranteeing that the leftist property holds when we create a new node.

**Lemma**: In a leftist heap with $n$ nodes, the rank of the root node is at most $\log_2 (n + 1).$
    
**Proof**: To prove this lemma, we first claim that if a heap has rank $r$, then it has at least $2^r - 1$ entries. 
Let $n(r)$ be the number of nodes in the smallest leftist heap of rank $r$. According to the way we assign rank, the right child of any node $x$ of rank $r$ is $r-1$. Also by the leftist property, for a node $x$ of rank $r$, $rank(L(x)) \geq rank(R(x)) = r - 1.$ Then For a leftist heap rooted at $x$ of rank $r$ we get the following recurrence:

$n(r) = 1 + n(L(x)) + n(R(x)) \geq 1 + 2n(r-1). $

Solving this recurrence yields that $n(r) = 2^{r}-1$. 

Now to prove the lemma, suppose we have a leftist heap with $n$ nodes and rank $r$. First, $n \geq n(r)$ by definition. This means that $n \geq 2^r - 1$ and so $r \leq \log_2 (n+1).

How does this help us?
    
**Theorem**: If $A$ and $B$ are leftist heaps, then the meld algorithm runs in $O(\log |A| + \log |B|)$ work and produces a leftist heap with all of the elements from $A$ and $B$.

**Proof**: The key observation is that the meld operation only advances along the right spine of either $A$ or $B$. But since rank decreases by 1 each time we advance, there are $rank(A) + rank(B)$ constant-time operations in total. By the lemma, this results in $O(\log |A| + \log |B|)$ work. 